In [4]:
from azureml.core import Workspace, Datastore
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.core import Dataset
from azure.ai.ml import MLClient
from azure.identity import InteractiveBrowserCredential

subscription_id = "0a94de80-6d3b-49f2-b3e9-ec5818862801"
resource_group = "buas-y2"
workspace_name = "Y2D-Staff-2025"

# Log in using interactive Auth
auth = InteractiveLoginAuthentication()

# Connect to workspace
workspace = Workspace(subscription_id=subscription_id,
                      resource_group=resource_group,
                      workspace_name=workspace_name,
                      auth=auth,
                      )

# Connect to the ML Client
credential = InteractiveBrowserCredential()

# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id=subscription_id,
    resource_group_name=resource_group,
    workspace_name=workspace_name,
)

Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented


In [5]:
# Check for existing compute targets
compute_targets = workspace.compute_targets

for name, ct in compute_targets.items():
    print(f'🖥️ {name}: {ct.type}, Provisioning state: {ct.provisioning_state}')

Class KubernetesCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


🖥️ adsai-lambda-0: Kubernetes, Provisioning state: Succeeded


In [6]:
# Check for existing compute targets in ML Client
print("Available compute targets:")
for compute in ml_client.compute.list():
    print(f'🖥️ {compute.name}: {compute.type}, Provisioning state: {compute.provisioning_state}')

Available compute targets:
🖥️ adsai-lambda-0: kubernetes, Provisioning state: Succeeded


In [3]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import DockerConfiguration

# Create a new environment
env = Environment(name="my-docker-environment")

# Create a base Docker image to use for the environment
env.docker.base_dockerfile = """
FROM ubuntu:18.04
RUN apt-get update && \
    apt-get install -y python3-pip && \
    pip3 install azureml-defaults
"""

# Create a new Conda dependencies object
conda_dep = CondaDependencies.create(python_version="3.8")

# Add some package dependencies
conda_dep.add_conda_package("numpy")
conda_dep.add_conda_package("pandas")
conda_dep.add_conda_package("scikit-learn")

# Add the dependencies to the environment
env.python.conda_dependencies = conda_dep

# Specify a Docker-based environment
docker_config = DockerConfiguration(use_docker=True)

# Register the environment to re-use later
env.register(workspace)

Property base_image is mutually exclusive with base_dockerfile. Reset base_image to None


{
    "assetId": "azureml://locations/westeurope/workspaces/db510a41-155d-4ea7-8569-34a7f7be6be0/environments/my-docker-environment/versions/1",
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": "\nFROM ubuntu:18.04\nRUN apt-get update &&     apt-get install -y python3-pip &&     pip3 install azureml-defaults\n",
        "baseImage": null,
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "buildContext": null,
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inf

In [7]:
# List all available environments.
envs = ml_client.environments.list()
for env in envs:
    print(f"Environment name: {env.name}, latest version: {env.latest_version}")

Environment name: AzureML-ACPT-pytorch-1.13-py38-cuda11.7-gpu, latest version: 10


In [ ]:
from azureml.core import Environment

custom_env_name = "training-environment-test"
version = "1.0.1"

# Create a custom environment.
env = Environment(
    name=custom_env_name,
    description="Custom environment for training with PyTorch",
    tags={"pytorch": "2.0"}, #for example, these just give you a way to tag your environments with metadata
    conda_file="conda.yaml",
    image= "mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:latest",
    version=version,
)

# Register the environment to the workspace.
env.register(workspace=workspace)

print(
    f"Environment with name {env.name} is registered to workspace, the environment version is {env.version}"
)


Environment with name training-environment-test is registered to workspace, the environment version is None


In [7]:
from azure.ai.ml.entities import Environment

custom_env_name = "training-environment-sdkv2"
version = "1.0.5"

# Create a custom environment using SDK v2
env = Environment(
    name=custom_env_name,
    description="Custom environment for training with PyTorch",
    tags={"pytorch": "2.0"},
    conda_file="conda.yaml",
    image="mcr.microsoft.com/aifx/acpt/stable-ubuntu2204-cu118-py310-torch260:biweekly.202505.2.v1",
    version=version,
)

# Register the environment to the workspace using ml_client
registered_env = ml_client.environments.create_or_update(env)

print(
    f"Environment with name {registered_env.name} is registered to workspace, the environment version is {registered_env.version}"
)

Environment with name training-environment-sdkv2 is registered to workspace, the environment version is 1.0.5


In [15]:
# List all available environments.
envs = ml_client.environments.list()
for env in envs:
    print(f"Environment name: {env.name}, latest version: {env.latest_version}")

Environment name: Example, latest version: 1
Environment name: training-environment-test, latest version: 1
Environment name: training-environment-sdkv2, latest version: 1.0.3
Environment name: training-environment, latest version: 1
Environment name: AzureML-ACPT-pytorch-1.13-py38-cuda11.7-gpu, latest version: 10
